In [1]:
import networkx as nx
import numpy as np
import requests
import zipfile
from io import BytesIO

In [2]:
## GET LOADED AND UNZIPPED ONLY CODE
link = "http://www-personal.umich.edu/~mejn/netdata/polblogs.zip"
data_dir = "data/political/raw/"

In [3]:
# Split URL to get the file name
filename = link.split('/')[-1]

In [4]:
# Downloading the file by sending the request to the URL
print('Beginning Political Data Download')
req = requests.get(link)
print('Political Data Download Completed')

Beginning Political Data Download
Political Data Download Completed


In [5]:
# extracting the zip file contents
zipfile= zipfile.ZipFile(BytesIO(req.content))
zipfile.extractall('../' + data_dir)

In [8]:
Z = nx.read_gml('../' + data_dir + "polblogs.gml")
Z

NetworkXError: edge #12757 (1047->1179) is duplicated.
Hint: If multigraph add "multigraph 1" to file header.

In [139]:
largest = max(nx.strongly_connected_components(H), key=len)

In [140]:
Largest_SCC = H.subgraph(largest)

In [141]:
Largest_SCC

In [109]:
def spectral_cluster(G):
    A = nx.to_numpy_array(G)
    D = np.diag(A.sum(axis=1))
    # graph laplacian
    L = D-A
    # eigenvalues and eigenvectors
    vals, vecs = np.linalg.eig(L)
    # sort these based on the eigenvalues
    vecs = vecs[:,np.argsort(vals)]
    vals = vals[np.argsort(vals)]
    return [1 if i > 0 else 0 for i in vals]

# Try on entire graph

In [127]:
predictions = spectral_cluster(H)

In [134]:
evaluate = []
index = 0
for i in list(H.nodes(data = True)):
    evaluate.append(predictions[index] == i[1]['value'])
    index += 1

In [136]:
sum(evaluate)/len(evaluate)

0.7778523489932886

# Try on strongest connected component

In [142]:
predictions = spectral_cluster(Largest_SCC)

In [143]:
evaluate = []
index = 0
for i in list(Largest_SCC.nodes(data = True)):
    evaluate.append(predictions[index] == i[1]['value'])
    index += 1

In [144]:
sum(evaluate)/len(evaluate)

0.5586380832282472